# Benchmark the reliability solvers on the problems

In this example, we show how to run all the methods on all the problems and get the computed probability.

In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd
import csv
from tqdm import tqdm
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from matplotlib import colors

We import the list of reliability problems.

In [2]:
benchmarkProblemList = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(benchmarkProblemList)
numberOfProblems

26

In [3]:
problem = benchmarkProblemList[15]
print(problem.getName())
event = problem.getEvent()
inputVector = event.getAntecedent()
distribution = inputVector.getDistribution()
dimension = distribution.getDimension()
print("Dim = %.d" % dimension)
print("Beta = %.2f" % problem.computeBeta())

RP89
Dim = 2
Beta = 2.55


In [4]:
problem_names = [
    benchmarkProblem.getName() for benchmarkProblem in benchmarkProblemList
]
columns = ["$d$", r"$p_f$", r"$\beta$"]
df_problems_list = pd.DataFrame(index=problem_names, columns=columns,)
for problem in benchmarkProblemList:
    name = problem.getName()
    d = problem.getEvent().getAntecedent().getDistribution().getDimension()
    pf = problem.getProbability()
    beta = problem.computeBeta()
    df_problems_list.loc[name] = [int(d), pf, beta]
pd.set_option("display.float_format", lambda x: "%.3e" % x)
df_problems_list

,$d$,$p_f$,$\beta$
RP8,6,7.898e-04,3.160e+00
RP14,5,7.520e-03,2.431e+00
RP22,2,4.160e-03,2.639e+00
RP24,2,2.860e-03,2.763e+00
RP25,2,6.140e-06,4.373e+00
RP28,2,1.453e-07,5.129e+00
RP31,2,1.800e-04,3.568e+00
RP33,3,2.570e-03,2.798e+00
RP35,2,3.540e-03,2.693e+00
RP38,7,8.100e-03,2.404e+00


In [5]:
latex_code = df_problems_list.to_latex()
text_file = open("reliability_problems_list.tex", "w")
text_file.write(latex_code)
text_file.close()

## Multi level columns: methods are level 1, metrics are level 2

For several algorithms and all the reliability problems, we want to estimate the failure probability and compare them.

In [6]:
problem_names = [
    benchmarkProblem.getName() for benchmarkProblem in benchmarkProblemList
]
methods = ["Monte Carlo", "FORM", "SORM", "FORM-IS", "Subset"]
metrics = ["Estimated Pf", "Correct digits", "Nb Simu."]
multi_columns = pd.MultiIndex.from_product(
    [methods, metrics], names=["Methods", "Metrics"]
)
multi_columns = multi_columns.insert(0, ("Exact", "Estimated Pf"))
df_multi = pd.DataFrame(
    np.zeros((len(problem_names), len(methods) * len(metrics) + 1)),
    index=problem_names,
    columns=multi_columns,
)

In [7]:
nearestPointAlgorithm = ot.AbdoRackwitz()
maximumOuterSampling = 10 ** 2
blockSize = 10 ** 2
coefficientOfVariation = 0.0

# TODO Make MC larger and investigate why are they so many zeros
for problem in tqdm(benchmarkProblemList):
    problem_name = problem.getName()
    metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)
    df_multi.loc[problem_name][("Exact", "Estimated Pf")] = problem.getProbability()
    # FORM
    benchmarkResult = metaAlgorithm.runFORM(nearestPointAlgorithm)
    pf, nb_digits, nb_simu = [
        benchmarkResult.computedProbability,
        benchmarkResult.numberOfCorrectDigits,
        benchmarkResult.numberOfFunctionEvaluations,
    ]
    df_multi.loc[
        problem_name,
        [("FORM", "Estimated Pf"), ("FORM", "Correct digits"), ("FORM", "Nb Simu.")],
    ] = [pf, abs(nb_digits), nb_simu]
    # SORM
    benchmarkResult = metaAlgorithm.runSORM(nearestPointAlgorithm)
    pf, nb_digits, nb_simu = [
        benchmarkResult.computedProbability,
        benchmarkResult.numberOfCorrectDigits,
        benchmarkResult.numberOfFunctionEvaluations,
    ]
    df_multi.loc[
        problem_name,
        [("SORM", "Estimated Pf"), ("SORM", "Correct digits"), ("SORM", "Nb Simu.")],
    ] = [pf, abs(nb_digits), nb_simu]
    # MC
    benchmarkResult = metaAlgorithm.runMonteCarlo(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    pf, nb_digits, nb_simu = [
        benchmarkResult.computedProbability,
        benchmarkResult.numberOfCorrectDigits,
        benchmarkResult.numberOfFunctionEvaluations,
    ]
    df_multi.loc[
        problem_name,
        [
            ("Monte Carlo", "Estimated Pf"),
            ("Monte Carlo", "Correct digits"),
            ("Monte Carlo", "Nb Simu."),
        ],
    ] = [pf, abs(nb_digits), nb_simu]
    # FORM-IS
    benchmarkResult = metaAlgorithm.runFORMImportanceSampling(
        nearestPointAlgorithm,
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    pf, nb_digits, nb_simu = [
        benchmarkResult.computedProbability,
        benchmarkResult.numberOfCorrectDigits,
        benchmarkResult.numberOfFunctionEvaluations,
    ]
    df_multi.loc[
        problem_name,
        [
            ("FORM-IS", "Estimated Pf"),
            ("FORM-IS", "Correct digits"),
            ("FORM-IS", "Nb Simu."),
        ],
    ] = [pf, abs(nb_digits), nb_simu]
    # Subset
    benchmarkResult = metaAlgorithm.runSubsetSampling(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    pf, nb_digits, nb_simu = [
        benchmarkResult.computedProbability,
        benchmarkResult.numberOfCorrectDigits,
        benchmarkResult.numberOfFunctionEvaluations,
    ]
    df_multi.loc[
        problem_name,
        [
            ("Subset", "Estimated Pf"),
            ("Subset", "Correct digits"),
            ("Subset", "Nb Simu."),
        ],
    ] = [pf, abs(nb_digits), nb_simu]
# df_multi

100%|██████████| 26/26 [00:10<00:00,  2.39it/s]


In [8]:
df_multi.style.format(
    "{:.2f}", subset=[(method, "Correct digits") for method in methods]
).format(
    "{:.3e}", subset=[(method, "Estimated Pf") for method in methods + ["Exact"]]
).format(
    "{:.0f}", subset=[(method, "Nb Simu.") for method in methods]
)

### Note: 

- Problems with the number of simulations on the subset simulation algorithm (already noticed)
- The control works for Monte Carlo, we are getting pf=0 because we are limiting ourself to 10^5 simulations for 10^-7 pf (RP 28, 107, 111, 77)
- FORM, SORM, FORM-IS are KO in front of some functions and stop after 1 iteration


## Correct digits table

In [9]:
# Extract the Correct digits columns and modify style
df_correct_digits = df_multi.loc[:, (methods, "Correct digits")].copy()
df_correct_digits.columns = methods

In [10]:
latex_code = df_correct_digits.to_latex()
text_file = open("correct_digits.tex", "w")
text_file.write(latex_code)
text_file.close()

In [11]:
df_correct_digits_styled = (
    df_correct_digits.style.format("{:.2f}")
    .background_gradient(cmap="Blues", axis=1)
    .set_properties(**{"color": "black"})
)
df_correct_digits_styled

,Monte Carlo,FORM,SORM,FORM-IS,Subset
RP8,0.86,0.78,2.11,1.69,0.91
RP14,0.06,0.04,0.04,0.05,0.05
RP22,0.80,0.31,1.26,1.62,1.99
RP24,0.59,0.07,0.07,1.41,0.88
RP25,1.18,2.53,0.14,0.82,0.66
RP28,0.00,0.09,0.00,1.08,0.68
RP31,1.07,2.10,2.10,1.24,1.32
RP33,0.42,0.32,0.32,1.02,1.24
RP35,0.82,0.21,0.40,0.48,1.51
RP38,1.31,1.61,2.06,2.25,1.51


In [12]:
f = open("correct_digits_table.html", "w")
f.write(df_correct_digits_styled.render())  # df is the styled dataframe
f.close()

## Add in the html file to change the font
"""
html *
{
   font-family: "Computer Modern Sans", sans-serif;
}
"""

'\nhtml *\n{\n   font-family: "Computer Modern Sans", sans-serif;\n}\n'

Adaptation de la fonction background_gradient avec une échelle log 

In [13]:
df = pd.DataFrame(
    [[66, 10000, 100, 10, 1], [666, 20000, 200, 20, 2], [6, 5000, 500, 50, 5]],
    columns=["a", "b", "c", "d", "e"],
)
df.style.background_gradient(cmap="Blues", axis=1)

,a,b,c,d,e
0,66,10000,100,10,1
1,666,20000,200,20,2
2,6,5000,500,50,5


In [14]:
def background_logscale_gradient(s, cmap="PuBu", low=0, high=0):
    rng = s.max() - s.min()
    # norm = colors.LogNorm(s.min() - (rng * low), s.max() + (rng * high))# Original
    norm = colors.LogNorm(s.min() + 1e-10, s.max())
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ["background-color: %s" % color for color in c]


test = df.style.apply(
    background_logscale_gradient, cmap="Blues", axis=1, subset=["b", "c", "d", "e"]
).set_properties(**{"color": "black"})
test

,a,b,c,d,e
0,66,10000,100,10,1
1,666,20000,200,20,2
2,6,5000,500,50,5


## Pf table

In [15]:
# Extract the Correct digits columns and modify style
df_pf = df_multi.loc[:, (["Exact"] + methods, "Estimated Pf")].copy()
df_pf.columns = ["Exact"] + methods

##df_pf.style.format("{:.3e}") # Version without background color modification
# df_pf.style.format("{:.3e}").background_gradient(cmap="Blues", axis=1) # unreadable with a linear scale
df_pf_styled = (
    df_pf.style.format("{:.3e}")
    .apply(background_logscale_gradient, cmap="Blues", axis=1)
    .set_properties(**{"color": "black"})
)
df_pf_styled

,Exact,Monte Carlo,FORM,SORM,FORM-IS,Subset
RP8,7.898e-04,9.000e-04,6.599e-04,7.837e-04,7.737e-04,8.863e-04
RP14,7.520e-03,9.000e-04,7.003e-04,6.988e-04,7.598e-04,8.720e-04
RP22,4.160e-03,3.500e-03,6.210e-03,4.391e-03,4.259e-03,4.117e-03
RP24,2.860e-03,3.600e-03,6.209e-03,6.209e-03,2.749e-03,2.486e-03
RP25,6.140e-06,1.000e-04,2.105e-03,1.064e-05,4.644e-05,3.415e-05
RP28,1.453e-07,0.000e+00,2.850e-08,0.000e+00,1.332e-07,1.756e-07
RP31,1.800e-04,2.300e-03,2.275e-02,2.275e-02,3.319e-03,3.919e-03
RP33,2.570e-03,1.600e-03,1.350e-03,1.350e-03,2.322e-03,2.718e-03
RP35,3.540e-03,3.000e-03,1.350e-03,2.134e-03,2.377e-03,3.430e-03
RP38,8.100e-03,8.500e-03,7.902e-03,8.029e-03,8.146e-03,7.848e-03


In [16]:
f = open("pf_table.html", "w")
f.write(df_pf_styled.render())  # df is the styled dataframe
f.close()

## Add in the html file to change the font

"""
html *
{
   font-family: "Computer Modern Sans", sans-serif;
}
"""

'\nhtml *\n{\n   font-family: "Computer Modern Sans", sans-serif;\n}\n'

## Run several algorithms on all problems and get detailed statistics

Run several algorithms on all reliability benchmark problems: print statistics on each problem.

In [17]:
def FormatRow(benchmarkResult):
    """Format a single row of the benchmark table"""
    result = [
        benchmarkResult.exactProbability,
        benchmarkResult.computedProbability,
        benchmarkResult.absoluteError,
        benchmarkResult.numberOfCorrectDigits,
        benchmarkResult.numberOfFunctionEvaluations,
        benchmarkResult.numberOfDigitsPerEvaluation,
    ]
    return result

In [18]:
method_names = ["Monte-Carlo", "FORM", "SORM", "FORM-IS", "SUBSET"]

maximumOuterSampling = 10 ** 2
blockSize = 10 ** 2
coefficientOfVariation = 0.0

for i in tqdm(range(numberOfProblems)):
    problem = benchmarkProblemList[i]
    name = problem_names[i]
    exact_pf_name = "%10s" % ("Exact PF " + name[0:10])
    metrics = [
        exact_pf_name,
        "Estimated PF",
        "Absolute Error",
        "Correct Digits",
        "Function Calls",
        "Digits / Evaluation",
    ]
    results = np.zeros((len(method_names), len(metrics)))
    metaAlgorithm = otb.ReliabilityBenchmarkMetaAlgorithm(problem)
    # Monte-Carlo
    benchmarkResult = metaAlgorithm.runMonteCarlo(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[0, :] = FormatRow(benchmarkResult)
    # FORM
    benchmarkResult = metaAlgorithm.runFORM(nearestPointAlgorithm)
    results[1, :] = FormatRow(benchmarkResult)
    # SORM
    benchmarkResult = metaAlgorithm.runSORM(nearestPointAlgorithm)
    results[2, :] = FormatRow(benchmarkResult)
    # FORM-IS
    benchmarkResult = metaAlgorithm.runFORMImportanceSampling(
        nearestPointAlgorithm,
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[3, :] = FormatRow(benchmarkResult)
    # Subset
    benchmarkResult = metaAlgorithm.runSubsetSampling(
        maximumOuterSampling=maximumOuterSampling,
        coefficientOfVariation=coefficientOfVariation,
        blockSize=blockSize,
    )
    results[4, :] = FormatRow(benchmarkResult)
    # Gather statistics and print them
    df = pd.DataFrame(results, index=method_names, columns=metrics,)
    # Format the columns for readability
    s = df.style.format(
        {
            exact_pf_name: lambda x: "{:.3e}".format(x),
            "Estimated PF": lambda x: "{:.3e}".format(x),
            "Absolute Error": lambda x: "{:.3e}".format(x),
            "Correct Digits": lambda x: "{:.1f}".format(abs(x)),
            "Function Calls": lambda x: "{:d}".format(int(x)),
            "Estimated PF": lambda x: "{:.2e}".format(x),
            "Digits / Evaluation": lambda x: "{:.1f}".format(abs(x)),
        }
    )
    display(s)

  0%|          | 0/26 [00:00<?, ?it/s]

,Exact PF RP8,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,7.898e-04,5.00e-04,2.898e-04,0.4,10000,0.0
FORM,7.898e-04,6.60e-04,1.299e-04,0.8,9,0.1
SORM,7.898e-04,7.84e-04,6.081e-06,2.1,9,0.2
FORM-IS,7.898e-04,8.17e-04,2.751e-05,1.5,10009,0.0
SUBSET,7.898e-04,7.68e-04,2.139e-05,1.6,40000,0.0


  4%|▍         | 1/26 [00:02<01:03,  2.53s/it]

,Exact PF RP14,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,7.520e-03,7.00e-04,6.820e-03,0.0,10000,0.0
FORM,7.520e-03,7.00e-04,6.820e-03,0.0,15,0.0
SORM,7.520e-03,6.99e-04,6.821e-03,0.0,15,0.0
FORM-IS,7.520e-03,7.52e-04,6.768e-03,0.0,10015,0.0
SUBSET,7.520e-03,8.64e-04,6.656e-03,0.1,40000,0.0


  8%|▊         | 2/26 [00:03<00:32,  1.34s/it]

...

,Exact PF Four-branc,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,2.186e-03,1.80e-03,3.860e-04,0.8,10000,0.0
FORM,2.186e-03,0.00e+00,2.186e-03,0.0,5,0.0
SORM,2.186e-03,0.00e+00,2.186e-03,0.0,5,0.0
FORM-IS,2.186e-03,0.00e+00,2.186e-03,0.0,5,0.0
SUBSET,2.186e-03,2.46e-03,2.720e-04,0.9,30000,0.0


,Exact PF R-S,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,7.865e-02,8.23e-02,3.650e-03,1.3,10000,0.0
FORM,7.865e-02,7.86e-02,4.163e-17,15.3,3,5.1
SORM,7.865e-02,7.86e-02,4.163e-17,15.3,3,5.1
FORM-IS,7.865e-02,7.80e-02,6.994e-04,2.1,10003,0.0
SUBSET,7.865e-02,7.76e-02,1.030e-03,1.9,20000,0.0


 96%|█████████▌| 25/26 [00:10<00:00,  2.42it/s]

,Exact PF Axial stre,Estimated PF,Absolute Error,Correct Digits,Function Calls,Digits / Evaluation
Monte-Carlo,2.920e-02,3.03e-02,1.102e-03,1.4,10000,0.0
FORM,2.920e-02,3.00e-02,7.846e-04,1.6,5,0.3
SORM,2.920e-02,2.93e-02,1.344e-04,2.3,5,0.5
FORM-IS,2.920e-02,2.98e-02,6.049e-04,1.7,10005,0.0
SUBSET,2.920e-02,3.05e-02,1.252e-03,1.4,20000,0.0


100%|██████████| 26/26 [00:10<00:00,  2.50it/s]
